# But where are my closest wineries?
#### Problem:  Oregon wine pass is great, all these wineries offer great deals, but crawling around their website to find the wineries that are closest sucks.  Also, wouldn't it be great to know the ratings before you went there?
1. Scrape Wine Passport site
2. Call Google API for map distances
3. todo: Get reviews from YELP
4. todo: Topic model reviews on YELP
5. todo: Return df with Name, address, distance, ratings, review topics



In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime, timedelta
import re

In [2]:
# i keep address and api key in separate config because privacy
with open("wine_config.json", "r") as f:
    wine_config = json.load(f)

# here's the OWP's handy dandy website
url = r'https://oregonwinepass.com/sponsor-type/wineries/'
api_base = 'https://maps.googleapis.com/maps/api/directions/json?'
my_addr = wine_config['address']
myk = wine_config['map_api']

##### Scrape with bs4

In [3]:
raw_html = get(url)
soup = BeautifulSoup(raw_html.content, 'html.parser')

In [4]:
wineries = soup.find_all("h3", class_="entry-title scroll-top")
addresses = soup.find_all("span", class_="addr")
winery_list = [x.contents[1].string.strip() for x in wineries]
addr_list = [x.contents[0].string.strip() for x in addresses]

In [5]:
details = soup.find_all("div", class_="details")
details_list = [x.findChildren("li") for x in details]
wine_dict = {}

##### WInery detials are a little funky since there are more than 1 for each winery

In [6]:
# that shib is burried in there, let's get her out jim
details_list[0][0].contents[0]

'2-for-1 Wine Tasting'

##### I'm certain there is a more efficient way of doing this

In [7]:
for i in range(len(winery_list)):
    sub_details = details_list[i]
    sub_list = [x.contents[0] for x in sub_details]
    wine_dict[winery_list[i]] = sub_list
    
wine_details_df = pd.DataFrame.from_dict(wine_dict, orient='index')
wine_details_df.head()    

0  \
AlexEli Vineyard                        2-for-1 Wine Tasting   
Angel Vine                       Bonus pour with Tasting Fee   
Anindor Vineyards                Complimentary tasting for 2   
ArborBrook Vineyards                         2 for 1 tasting   
Archer Vineyard       2 for 1 tasting (regularly $20/person)   

                                                                      1  \
AlexEli Vineyard                              10% off 4 Bottle Purchase   
Angel Vine                                 10% off all bottle purchases   
Anindor Vineyards                                10% off wine purchases   
ArborBrook Vineyards                          10% off on wine purchases   
Archer Vineyard       waived tasting fee with a purchase of any 4 wi...   

                                                        2     3  
AlexEli Vineyard                                     None  None  
Angel Vine            15% off case purchase – mix & match  None  
Anindor Vineyards                                    None  None  
ArborBrook Vineyards                                 None  None  
Archer Vineyard                20% off $100 wine purchase  None

##### gut check we didn't mess anything up thus far

In [8]:
assert len(wine_details_df) == len(addr_list), 'you done did messed up'
# thank god these are in order, amirite?
wine_details_df['address'] = addr_list

In [9]:
wine_details_df.head()    

0  \
AlexEli Vineyard                        2-for-1 Wine Tasting   
Angel Vine                       Bonus pour with Tasting Fee   
Anindor Vineyards                Complimentary tasting for 2   
ArborBrook Vineyards                         2 for 1 tasting   
Archer Vineyard       2 for 1 tasting (regularly $20/person)   

                                                                      1  \
AlexEli Vineyard                              10% off 4 Bottle Purchase   
Angel Vine                                 10% off all bottle purchases   
Anindor Vineyards                                10% off wine purchases   
ArborBrook Vineyards                          10% off on wine purchases   
Archer Vineyard       waived tasting fee with a purchase of any 4 wi...   

                                                        2     3  \
AlexEli Vineyard                                     None  None   
Angel Vine            15% off case purchase – mix & match  None   
Anindor Vineyards                                    None  None   
ArborBrook Vineyards                                 None  None   
Archer Vineyard                20% off $100 wine purchase  None   

                                                        address  
AlexEli Vineyard              35803 South Highway 213 , Molalla  
Angel Vine                         2025 SE 7th Avenue, Portland  
Anindor Vineyards                       1171 Vintage Dr, Elkton  
ArborBrook Vineyards            17770 NE Calkins Lane , Newberg  
Archer Vineyard       32230 NE Old Parrett Mountain Rd, Newberg

In [10]:
def replace_reserved(string):
    '''remove reserved url/uri characters'''
    for char in [';' , '/' , '?' , ':' , '@' , '&' , '=' , '+' , '$' , ',' , '#']:
        if char in string:
            string = string.replace(char, '')
    
    return string
    
def time_string(string):
    '''deal with google apis deal of formatting times nicely for humans'''
    if 'hours' in string and 'mins' in string:
        t = datetime.strptime(string, '%H hours %M mins')
    
    elif 'hour' not in string and 'mins' in string:
        t = datetime.strptime(string, '%M mins') 
    elif 'mins' in string:
        t = datetime.strptime(string, '%H hour %M mins')
    
    elif 'hours' in string:
        t = datetime.strptime(string, '%H hours %M min')
    else:
        t = datetime.strptime(string, '%H hour %M min')
    
    time_str = str(timedelta(hours=t.hour, minutes=t.minute, seconds=t.second))
    
    return time_str
    
    
def route_data(address):
    '''return distance and formatted duration a given address'''
    address = replace_reserved(address)
    dest = (address.replace(' ', '+'))
    api_str = api_base + 'origin=' + dest + '&destination=' + my_addr + '+Oregon' + '&key=' + myk
    # call api
    resp = get(api_str)
    json_resp = resp.json()
    dist = json_resp['routes'][0]['legs'][0]['distance']['text']
    duration = json_resp['routes'][0]['legs'][0]['duration']['text']
    dura = time_string(duration)
    
    return (dist, dura)


##### Loop through all address and return distance and duration in lists

In [11]:
dists = []
durations = []

for address in wine_details_df['address']:
    dist, duration = route_data(address)
    dists.append(dist)
    durations.append(duration)
    

##### Add distrance and duration to dataframe, sort

In [12]:
wine_details_df['duration'] = durations
wine_details_df['distances'] = dists
wine_details_df.sort_values(by='duration', inplace=True)

wine_details_df

0  \
Namaste Vineyards                                                            2-for-1 Tasting   
Willamette Valley Vineyards                                      Complimentary tasting for 2   
Johan Vineyards                                                  Complimentary tasting for 2   
Coria Estates                                                    Complimentary Tasting for 2   
Van Duzer Vineyards                                                          2 for 1 tasting   
Pfeiffer Winery                            enjoy 2 complimentary pinot plus tastings in t...   
Silas Wines                                                                  2 for 1 tasting   
Franny Beck Wines                          Complimentary tasting for two with a one bottl...   
Coelho Winery                                                                2 for 1 tasting   
Mystic Wine                                                                  2-for-1 Tasting   
Grochau Cellars                                                              2-for-1 Tasting   
Naked Winery- McMinnville                                        Complimentary tasting for 2   
J. Scott Cellars                                                             2 for 1 tasting   
Elizabeth Chambers Cellar                                                    2 for 1 tasting   
Terra Vina                                                                   2 for 1 Tasting   
Yamhill Valley Vineyards                                                    2 for 1 tastings   
Walnut City WineWorks                                Enjoy a complimentary flight of 5 wines   
Vitis Ridge                                                               10% off any bottle   
Aurora Cellars at Aurora Colony Vineyards                                               None   
Silvan Ridge Winery                              Complimentary tastings for passport members   
Saginaw Vineyard                           Complimentary tasting for passport members and...   
Hyland Estates                                                              2 for 1 Tastings   
Troon Vineyard – Carlton                                         Complimentary tasting for 2   
Stone Griffon Vineyard                                                       2 for 1 tasting   
Gypsy Dancer Wines                                                           2-for-1 tasting   
Hanson Vineyards                                                            2 for 1 tastings   
Carlton Cellars                                                 Complimentary tastings for 2   
Longplay Wine                                                    Complimentary tasting for 2   
Cliff Creek Cellars-Newberg                                                 2 for 1 Tastings   
Vintyr Wine                                                             2-for-1 wine tasting   
...                                                                                      ...   
Angel Vine                                                       Bonus pour with Tasting Fee   
Forest Edge Vineyard                              Complimentary tasting for Passport Members   
SakéOne                                    Two complimentary Toji Flights (flight of 6 in...   
Blooming Hill Vineyard                                                      2 for 1 tastings   
David Hill Vineyard & Winery                                                2 for 1 Tastings   
D’ Anu Wines                                                     Bonus pour with Tasting Fee   
Risdall Ranch Winery                                                   2 for 1 wine tastings   
Bradley Vineyards                                                Complimentary tasting for 2   
River’s Edge Winery                                              Complimentary tasting for 2   
Anindor Vineyards                                                Complimentary tasting for 2   
Buddha Kat Winery                          complimentary ta

In [17]:
wine_details_df.to_csv('oregon_wineries.csv')

# Yelp scraper